# Мазмұнға негізделген сүзгілеу үшін терең үйрену

Осы жаттығуда, сіз нейрондық желіні пайдаланып, фильмдерге арналған ұсыныс жүйесін құру үшін мазмұнға негізделген сүзгілеуді жүзеге асырасыз. Сіздің тапсырмаңыз соңында беріледі.

Біз таныс пакеттерді, яғни NumPy, TensorFlow және [scikit-learn](https://scikit-learn.org/stable/)-нің пайдалы функцияларын қолданамыз. Сондай-ақ, кестелерді әдемі түрде шығару үшін [tabulate](https://pypi.org/project/tabulate/) және кестелік деректерді ұйымдастыру үшін [Pandas](https://pandas.pydata.org/) пайдаланамыз.

In [62]:
import numpy as np
import numpy.ma as ma
from numpy import genfromtxt
from collections import defaultdict
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
from recsysNN_utils import *
pd.set_option("display.precision", 1)
from IPython.display import HTML

## Фильм бағалау деректері  
Деректер жиынтығы [MovieLens ml-latest-small](https://grouplens.org/datasets/movielens/latest/) жинағынан алынған.

Бастапқы деректер жиынтығында 600 пайдаланушы тарапынан бағаланған 9000 фильм бар, бағалау 0.5-тен 5-ке дейінгі шкала бойынша, 0.5 қадамдармен берілген. 2000 жылдан бастап шыққан және танымал жанрлардағы фильмдерге назар аудару мақсатында, деректер жиынтығы кішірейтілді. Кішірейтілген жиынтықта 395 пайдаланушы және 694 фильм бар. Әр фильм үшін жиынтықта фильм атауы, шыққан жылы және бір немесе бірнеше жанр беріледі. Мысалы, *Toy Story 3* фильмі 2010 жылы шыққан және бірнеше жанрға ие: "Adventure|Animation|Children|Comedy|Fantasy|IMAX". Бұл жиынтық пайдаланушылар туралы бағалауларынан басқа өте аз ақпарат қамтиды және төменде сипатталған нейрондық желілерді оқыту векторларын жасау үшін қолданылады.

### Нейрондық желімен мазмұнға негізделген сүзгілеу

Бірлескен сүзгілеу зертханасында, сіз пайдаланушы векторын және фильм/зат векторын генерацияладыңыз, олардың нүктелік көбейтіндісі бағаны болжау үшін қолданылды. Бұл векторлар тек бағалаулардан алынған болатын.

Мазмұнға негізделген сүзгілеу де пайдаланушы және фильмнің ерекшелік векторын жасайды, бірақ пайдаланушы мен/немесе фильм туралы болжауды жақсартатын қосымша ақпараттың да болуы мүмкін екенін мойындайды. Қосымша ақпарат нейрондық желіге беріледі, ол пайдаланушы мен фильм векторын генерациялайды.

Желідегі фильм контенті бастапқы деректер мен кейбір «инженерлік ерекшеліктердің» комбинациясынан тұрады. Бастапқы ерекшеліктерге фильмнің шыққан жылы және бір ыстық вектор түрінде ұсынылған фильмнің жанры кіреді. 14 жанр бар. Инженерлік ерекшелік – пайдаланушы бағаларынан алынған орташа баға. Бірнеше жанры бар фильмдер әр жанр үшін оқыту векторын алады.

Пайдаланушы контенті тек инженерлік ерекшеліктерден құралады. Әр пайдаланушы үшін жанр бойынша орташа баға есептеледі. Сонымен қатар, пайдаланушы идентификаторы, баға саны және орташа баға бар, бірақ олар оқыту немесе болжау контентіне қосылмайды, олар деректерді түсіндіруде пайдалы.

Оқыту жиынтығы деректер жиынтығындағы барлық пайдаланушы бағаларынан тұрады. Пайдаланушы және фильм/зат векторлары жоғарыда көрсетілген желіге оқыту жиынтығы ретінде бірге беріледі. Пайдаланушы векторы пайдаланушы берген барлық фильмдер үшін бірдей.

Төменде, кейбір деректерді жүктеп, көрсетейік.


In [63]:
# пайдаланушы мен фильмге қатысты деректерді дайындап, модельге беруге дайын күйге келтіру
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

# 🔹 Ерекшелік өлшемдерін орнату
num_user_features = user_train.shape[1] - 3  # ID, рейтинг саны, орташа рейтингті алып тастаймыз
num_item_features = item_train.shape[1] - 1  # Тек фильм ID-ін алып тастаймыз

# 🔹 Жанр мен ерекшелік векторларының басталу индекстері
uvs = 3  # Пайдаланушы жанр векторының басталуы
ivs = 3  # Фильм жанр векторының басталуы
u_s = 3  # Пайдаланушы ерекшеліктері басталатын баған
i_s = 1  # Фильм ерекшеліктері басталатын баған

scaledata = True  # Масштабтау қолдану керек пе

print(f"Оқыту векторларының саны: {len(item_train)}")

Оқыту векторларының саны: 58187


Пайдаланушы және элемент/фильм ерекшеліктерінің кейбіреулері оқыту кезінде пайдаланылмайды. Төменде жақшадағы "[]" ішіндегі, мысалы, "пайдаланушы идентификаторы", "бағалау саны" және "орташа баға" сияқты ерекшеліктер модель оқытылып қолданылғанда қосылмайды. Назар аударыңыз, барлық бағаланған фильмдер үшін пайдаланушы векторы бірдей.

In [64]:
HTML(pprint_train(user_train, user_features, uvs,  u_s, maxcount=5))

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9


In [65]:
HTML(pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False))

[movie id],year,ave rating,Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
6874,2003,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6874,2003,4.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6874,2003,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8798,2004,3.8,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8798,2004,3.8,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [66]:
print(f"y_train[:5]: {y_train[:5]}")

y_train[:5]: [4.  4.  4.  3.5 3.5]


Жоғарыда 6874 нөмірлі фильмнің 2003 жылы шыққан экшн фильмі екені көрініп тұр. Пайдаланушы 2 экшн фильмдерді орта есеппен 3.9 бағалайды. Сонымен қатар, 6874 нөмірлі фильм қылмыс және триллер жанрларында да тіркелген. MovieLens пайдаланушылары бұл фильмге орташа 4 баға берген. Оқыту үлгісі екі кестеден бір жол және y_train бағанындағы бағадан тұрады.

In [67]:
# масштабты оқыту деректері Масштабтау (StandardScaler) — модельге кіретін деректердің мәнін бірдей шкалаға келтіру үшін қажет (орташа мәні 0, стандартты ауытқу 1 болады).
if scaledata:  # Егер масштабтау қосылған болса (True болса)
    item_train_save = item_train  # Бастапқы фильм деректерін сақтап аламыз
    user_train_save = user_train  # Бастапқы пайдаланушы деректерін де сақтап аламыз

    scalerItem = StandardScaler()  # Фильм векторларын масштабтау үшін скейлер
    scalerItem.fit(item_train)  # Фильм векторларының орташа мәні мен стандартты ауытқуын есептейді
    item_train = scalerItem.transform(item_train)  # Масштабталған фильм векторларын аламыз

    scalerUser = StandardScaler()  # Пайдаланушы векторларын масштабтау үшін скейлер
    scalerUser.fit(user_train)  # Пайдаланушы векторлары үшін масштабтау параметрлерін есептейді
    user_train = scalerUser.transform(user_train)  # Масштабталған пайдаланушы векторлары

    # Масштабталған деректерді кері айналдырсақ, бастапқы деректерге жақын болуын тексеру
    print(np.allclose(item_train_save, scalerItem.inverse_transform(item_train)))  
    print(np.allclose(user_train_save, scalerUser.inverse_transform(user_train)))  


True
True


Нәтижелерді бағалау үшін, деректерді Course 2, апта 3-тен талқыланғандай оқыту және тест жиынтықтарына бөлеміз. Мұнда деректерді бөлу және араластыру үшін [sklearn train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) қолданамыз. Бастапқы кездейсоқ күйді бірдей мәнге орнату item, user және y элементтерінің бірдей түрде араластырылуын қамтамасыз етеді.

In [68]:
# Деректерді оқыту және тест жиынтығына бөлу
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)

# Бөлінген деректердің өлшемін шығару
print(f"фильм/зат оқыту деректерінің өлшемі: {item_train.shape}")
print(f"фильм/зат тест деректерінің өлшемі: {item_test.shape}")


фильм/зат оқыту деректерінің өлшемі: (46549, 17)
фильм/зат тест деректерінің өлшемі: (11638, 17)


Масштабталған, араластырылған деректердің орташа мәні енді нөлге тең.

In [69]:
HTML(pprint_train(user_train, user_features, uvs, u_s, maxcount=5))

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
1,0,0.6,0.7,0.6,0.6,0.7,0.7,0.5,0.7,0.2,0.3,0.3,0.5,0.5,0.8,0.5
0,0,1.6,1.5,1.7,0.9,1.0,1.4,0.8,-1.2,1.2,1.2,1.6,0.9,1.4,1.2,1.0
0,0,0.8,0.6,0.7,0.5,0.6,0.6,0.3,-1.2,0.7,0.8,0.9,0.6,0.2,0.6,0.6
1,0,-0.1,0.2,-0.1,0.3,0.7,0.3,0.2,1.0,-0.5,-0.7,-2.1,0.5,0.7,0.3,0.0
-1,0,-1.3,-0.8,-0.8,0.1,-0.1,-1.1,-0.9,-1.2,-1.5,-0.6,-0.5,-0.6,-0.9,-0.4,-0.9


Мақсатты бағалауларды Min Max Scaler арқылы -1 мен 1 аралығында масштабтаңыз. Біз scikit-learn қолданамыз, өйткені оның inverse_transform функциясы бар. [scikit learn MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

In [70]:
scaler = MinMaxScaler((-1, 1))  # Бағаларды -1 мен 1 аралығына шкалалау үшін MinMaxScaler құрылады
scaler.fit(y_train.reshape(-1, 1))  # y_train-ді (бағаларды) шкалалауға дайындау (reshape арқылы баған түріне келтіреміз)
ynorm_train = scaler.transform(y_train.reshape(-1, 1))  # Шкалаланған оқыту бағалары
ynorm_test = scaler.transform(y_test.reshape(-1, 1))  # Шкалаланған тест бағалары
print(ynorm_train.shape, ynorm_test.shape)  # Нәтиже өлшемін шығару


(46549, 1) (11638, 1)


## Мазмұнға негізделген сүзгілеу үшін нейрондық желі

Енді жоғарыда көрсетілген суретте сипатталғандай нейрондық желіні құрамыз. Ол екі желіден тұрады, олар нүктелік көбейтінді арқылы біріктіріледі. Сіз осы екі желіні құрастырасыз. Бұл мысалда олар бірдей болады. Алайда, осы желілер бірдей болу қажет емес. Егер пайдаланушы контенті фильм контентінен едәуір үлкен болса, сіз пайдаланушы желісінің күрделілігін фильм желісіне қарағанда арттыруды таңдай аласыз. Бұл жағдайда контент ұқсас болғандықтан, желілер бірдей.

- Keras Sequential моделін қолданыңыз:
    - Бірінші қабат – 256 бірлікті және relu активациясын қолданатын тығыз қабат.
    - Екінші қабат – 128 бірлікті және relu активациясын қолданатын тығыз қабат.
    - Үшінші қабат – `num_outputs` бірлікті және сызықтық немесе активациясыз тығыз қабат.
    
Желі қалғасын кодпен қамтамасыз етіледі. Қамтамасыз етілген код Keras Sequential моделін емес, Keras [Functional API](https://keras.io/guides/functional_api/) моделін қолданады. Бұл формат компоненттердің өзара байланысын икемді түрде құруға мүмкіндік береді.

In [71]:
# Пайдаланушының нейрондық желісі
num_outputs = 32  # соңғы қабатта шығатын нейрондар саны
tf.random.set_seed(1)  # Тұрақты кездейсоқтық үшін бастапқы егу

# Пайдаланушының нейрондық желісі: 256 және 128 нейрондары бар қабаттардан тұрады
user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),  # бірінші қабат
    tf.keras.layers.Dense(128, activation='relu'),  # екінші қабат
    tf.keras.layers.Dense(num_outputs, activation='linear'),  # соңғы қабат, сызықтық активация
])

# Фильмдердің нейрондық желісі
item_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),  # бірінші қабат
    tf.keras.layers.Dense(128, activation='relu'),  # екінші қабат
    tf.keras.layers.Dense(num_outputs, activation='linear'),  # соңғы қабат, сызықтық активация
])

# Пайдаланушының кірісін алу және негізгі желіге жіберу
input_user = tf.keras.layers.Input(shape=(num_user_features,))  # пайдаланушының ерекшеліктері
vu = user_NN(input_user)  # пайдаланушының ерекшеліктері бойынша нейрондық желіге бағыттау
vu = tf.keras.layers.Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vu)  # L2 нормализациясы (вектордың ұзындығын 1-ге келтіру)

# Фильмнің кірісін алу және негізгі желіге жіберу
input_item = tf.keras.layers.Input(shape=(num_item_features,))  # фильмнің ерекшеліктері
vm = item_NN(input_item)  # фильмнің ерекшеліктері бойынша нейрондық желіге бағыттау
vm = tf.keras.layers.Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vm)  # L2 нормализациясы

# Пайдаланушы мен фильмнің ерекшелік векторларының нүктелік көбейтіндісін есептеу
output = tf.keras.layers.Dot(axes=1)([vu, vm])  # екі вектор арасындағы ұқсастық

# Модельді анықтау: кіріс ретінде пайдаланушы және фильм, шығыс ретінде ұқсастық (нүктелік көбейтінді)
model = Model([input_user, input_item], output)

model.summary()  # Модельдің құрылымын шығару


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_2        │ (None, 32)        │     40,864 │ input_layer_5[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_3        │ (None, 32)        │     41,376 │ input_layer_7[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_3 (Lambda)   │ (None, 32)        │          0 │ sequential_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_4 (Lambda)   │ (None, 32)        │          0 │ sequential_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_5 (Dot)         │ (None, 1)         │          0 │ lambda_3[0][0],   │
│                     │                   │            │ lambda_4[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 82,240 (321.25 KB)

 Trainable params: 82,240 (321.25 KB)

 Non-trainable params: 0 (0.00 B)

Біз орташа квадраттық қате шығын функциясын және Adam оптимизаторын қолданамыз.

In [86]:
# Жасанды нейрондық желі үшін кездейсоқтықты тұрақтандыру
tf.random.set_seed(1)  # Тұрақты кездейсоқтықты қамтамасыз ету үшін бастапқы егу

# Құрылатын модель үшін шығын функциясын (loss function) анықтау
cost_fn = tf.keras.losses.MeanSquaredError()  # Орташа квадраттық қате (Mean Squared Error) шығын функциясы

# Оптимизаторды анықтау
opt = keras.optimizers.Adam(learning_rate=0.01)  # Адам (Adam) оптимизаторы, оқу жылдамдығы 0.01

# Модельді компиляциялау
model.compile(optimizer=opt,  # Оптимизаторды таңдау
              loss=cost_fn)   # Шығын функциясын таңдау


In [73]:
# Тұрақты кездейсоқтықты орнату үшін бастапқы егу
tf.random.set_seed(1)  # Тұрақты нәтижелер алу үшін кездейсоқтықты бастау кезінде орнатамыз

# Модельді оқыту
model.fit([user_train[:, u_s:], item_train[:, i_s:]], ynorm_train, epochs=30)  # Модельді оқыту

Epoch 1/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.1302
Epoch 2/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1172
Epoch 3/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.1151
Epoch 4/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1136
Epoch 5/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.1125
Epoch 6/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.1116
Epoch 7/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1108
Epoch 8/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1100
Epoch 9/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1092
Epoch 10/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1081
Epoch 11/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.1069
Epoch 12/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1062
Epoch 13/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.1055
Epoch 14/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.1049
Epoch 15/30
1455/1455 ━━━━━━━

Модельді тест деректері бойынша жоғалтуды анықтау үшін бағалаңыз. Ол оқыту жоғалтумен салыстырмалы, яғни модель оқыту деректеріне айтарлықтай артық бейімделмегенін көрсетеді.

In [74]:
# Модельді тест деректері бойынша бағалау
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], ynorm_test)


364/364 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1066


0.10644135624170303

### 3.1 Болжаулар
Төменде сіз моделі арқылы әртүрлі жағдайларда болжаулар жасайсыз.

#### Жаңа пайдаланушыға арналған болжаулар
Алдымен, біз жаңа пайдаланушы жасап, сол пайдаланушыға фильмдерді ұсынуды модельден сұраймыз. Мысал пайдаланушы контентін қолданып көргеннен кейін, өз қалауыңызға сәйкес пайдаланушы контентін өзгертіп, модельдің не ұсынатынын тексеруге болады. Ескеріңіз, бағалар 0.5-тен 5.0-ға дейінгі диапазонда, жарты қадамдық ұлғайту арқылы беріледі.

In [75]:
#жаңа пайдаланушының мәліметтерін енгізуге арналған
new_user_id = 5000  # Жаңа пайдаланушының идентификаторы
new_rating_ave = 1.0  # Пайдаланушының орташа бағасы
new_action = 1.0  # Әрекет жанрына арналған мән
new_adventure = 1  # Приключение жанрына арналған мән
new_animation = 1  # Анимация жанрына арналған мән
new_childrens = 1  # Балаларға арналған фильмдер жанрына арналған мән
new_comedy = 5  # Комедия жанрына арналған мән
new_crime = 1  # Қылмыс жанрына арналған мән
new_documentary = 1  # Документалды фильмдер жанрына арналған мән
new_drama = 1  # Драма жанрына арналған мән
new_fantasy = 1  # Фэнтези жанрына арналған мән
new_horror = 1  # Қорқынышты жанрға арналған мән
new_mystery = 1  # Мистика жанрына арналған мән
new_romance = 5  # Романтика жанрына арналған мән
new_scifi = 5  # Ғылым-фантастика жанрына арналған мән
new_thriller = 1  # Триллер жанрына арналған мән
new_rating_count = 3  # Пайдаланушының берген рейтингтер саны

# Пайдаланушының барлық деректерін бір массивке жинақтау
user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])


Жаңа пайдаланушы үшін ең жоғары бағаланған фильмдерді қарастырайық. Ескеріңіз, пайдаланушы векторы комедия және романтикалық жанрларға басымдық берген болатын.  
Төменде оқыту/тест жиынтығындағы әр фильмге арналған векторлары бар `item_vecs` жиынтығын қолданамыз. Бұл векторлар жоғарыда берілген пайдаланушы векторымен сәйкестендіріліп, масштабталған векторлар жаңа пайдаланушыға арналған барлық фильмдерге бағаларды болжау үшін пайдаланылады.

In [76]:
# деректер жиынындағы фильмдер санына сәйкес пайдаланушы векторын жасап, қайталау.
user_vecs = gen_user_vecs(user_vec, len(item_vecs))

# векторларды масштабтап, барлық фильмдер үшін болжау жасау. Нәтижелерді баға бойынша сұрыпталған түрде қайтарады.
sorted_index, sorted_ypu, sorted_items, sorted_user = predict_uservec(user_vecs, item_vecs, model, u_s, i_s,
                                                                       scaler, scalerUser, scalerItem, scaledata=scaledata)

HTML(print_pred_movies(sorted_ypu, sorted_user, sorted_items, movie_dict, maxcount=10))

59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


y_p,movie id,rating ave,title,genres
4.8947,62155,3.35,Nick and Norah's Infinite Playlist (2008),Comedy|Drama|Romance
4.88779,58047,3.42857,"Definitely, Maybe (2008)",Comedy|Drama|Romance
4.88345,56949,3.3,27 Dresses (2008),Comedy|Romance
4.88255,54004,3.45455,I Now Pronounce You Chuck and Larry (2007),Comedy|Romance
4.88034,41285,3.44118,Match Point (2005),Crime|Drama|Romance
4.8692,27808,3.36364,Spanglish (2004),Comedy|Drama|Romance
4.86815,76293,3.31818,Date Night (2010),Action|Comedy|Romance
4.86439,48043,3.5,"Fountain, The (2006)",Drama|Fantasy|Romance
4.85919,34162,3.50862,Wedding Crashers (2005),Comedy|Romance
4.85855,7151,3.375,Girl with a Pearl Earring (2003),Drama|Romance


Егер жоғарыда пайдаланушы векторын құрсаңыз, желі пайдаланушы жанрларының **жиынтығын** қамтитын вектор негізінде бағаны болжауға үйретілгенін ескеру маңызды. Тек бір жанрға ең жоғары баға беріп, қалған жанрларға ең төмен баға беру желі үшін мағыналы болмауы мүмкін, егер оқыту деректерінде мұндай баға жиынтығы бар пайдаланушылар болмаған болса.

#### Бар пайдаланушы үшін болжаулар  
"Пайдаланушы 36" — деректер жиынындағы пайдаланушылардың бірі — үшін болжауларға назар аударайық. Біз болжауларды модель бағаларымен салыстыра аламыз. Назар аударыңыз, бірнеше жанрға жататын фильмдер оқыту деректерінде бірнеше рет кездеседі. Мысалы, *The Time Machine* фильмі үш жанрға жатады: **Adventure**, **Action**, **Sci-Fi**.

In [77]:
uid = 36
# пайдаланушы векторларының жиынын қалыптастыру. Бұл бірдей вектор, түрлендірілген және қайталанған.
user_vecs, y_vecs = get_user_vecs(uid, scalerUser.inverse_transform(user_train), item_vecs, user_to_genre)

# векторларды масштабтап, барлық фильмдер үшін болжау жасау. Нәтижелерді баға бойынша сұрыпталған түрде қайтарады.
sorted_index, sorted_ypu, sorted_items, sorted_user = predict_uservec(user_vecs, item_vecs, model, u_s, i_s, scaler,
                                                                      scalerUser, scalerItem, scaledata=scaledata)
sorted_y = y_vecs[sorted_index]

# сұрыпталған болжауларды басып шығару
HTML(print_existing_user(sorted_ypu, sorted_y.reshape(-1,1), sorted_user, sorted_items, item_features, ivs, uvs, movie_dict, maxcount=10))


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


y_p,y,user,user genre ave,movie rating ave,title,genres
2.8,3.0,36,3.00,2.86,"Time Machine, The (2002)",Adventure
2.7,3.0,36,3.00,2.86,"Time Machine, The (2002)",Sci-Fi
2.7,3.0,36,3.00,2.86,"Time Machine, The (2002)",Action
1.8,1.0,36,1.50,4.00,"Beautiful Mind, A (2001)",Drama
1.6,1.0,36,1.00,4.00,"Beautiful Mind, A (2001)",Romance
1.6,1.5,36,1.75,3.52,Road to Perdition (2002),Crime
1.6,2.0,36,1.75,3.52,Gangs of New York (2002),Crime
1.3,1.5,36,1.50,3.52,Road to Perdition (2002),Drama
1.3,2.0,36,1.50,3.52,Gangs of New York (2002),Drama


#### Ұқсас элементтерді табу  
Жоғарыдағы нейрондық желі екі ерекшелік векторын шығарады: пайдаланушы ерекшелік векторы $v_u$ және фильм ерекшелік векторы $v_m$. Бұл — мағынасын түсіндіру қиын 32 өлшемді векторлар. Алайда, ұқсас элементтердің векторлары да ұқсас болады. Бұл ақпаратты ұсыным жасау үшін пайдалануға болады. Мысалы, егер пайдаланушы "Toy Story 3" фильміне жоғары баға берген болса, ұқсас фильм ерекшелік векторлары бар басқа фильмдерді ұсынып көруге болады.

Ұқсастықты өлшеудің бір тәсілі — екі вектордың $ \mathbf{v_m^{(k)}}$ және $\mathbf{v_m^{(i)}}$ арасындағы квадраттық қашықтықты есептеу:
$$\left\Vert \mathbf{v_m^{(k)}} - \mathbf{v_m^{(i)}}  \right\Vert^2 = \sum_{l=1}^{n}(v_{m_l}^{(k)} - v_{m_l}^{(i)})^2\tag{1}$$

### Квадраттық қашықтықты есептейтін функция.

In [78]:
def sq_dist(a, b):
    """
    Екі вектор арасындағы квадраттық қашықтықты қайтарады.

    Параметрлер:
      a (ndarray (n,)): n ерекшелігі бар вектор
      b (ndarray (n,)): n ерекшелігі бар вектор

    Қайтарымы:
      d (float): қашықтық
    """
    d = sum(np.square(a - b))
    return d

In [79]:
a1 = np.array([1.0, 2.0, 3.0]); b1 = np.array([1.0, 2.0, 3.0])
a2 = np.array([1.1, 2.1, 3.1]); b2 = np.array([1.0, 2.0, 3.0])
a3 = np.array([0, 1, 0]);       b3 = np.array([1, 0, 0])

print(f"a1 мен b1 арасындағы квадраттық қашықтық: {sq_dist(a1, b1)}")
print(f"a2 мен b2 арасындағы квадраттық қашықтық: {sq_dist(a2, b2)}")
print(f"a3 мен b3 арасындағы квадраттық қашықтық: {sq_dist(a3, b3)}")


a1 мен b1 арасындағы квадраттық қашықтық: 0.0
a2 мен b2 арасындағы квадраттық қашықтық: 0.030000000000000054
a3 мен b3 арасындағы квадраттық қашықтық: 2


Фильмдер арасындағы қашықтықтар матрицасын модель бір рет оқытылғаннан кейін есептеп, оны қайта оқытпай-ақ жаңа ұсынымдар үшін қайта пайдалануға болады. Модель оқытылып болғаннан кейінгі алғашқы қадам — әр фильм үшін фильм ерекшелік векторын, яғни $v_m$ алуға кірісу. Бұл үшін біз оқытылған `item_NN` желісін пайдаланамыз және фильм векторларын өткізіп, $v_m$ генерациялау үшін шағын қосымша модель құрамыз.

In [80]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda

input_item_m = Input(shape=(num_item_features,))     # енгізу қабаты
vm_m = item_NN(input_item_m)                         # оқытылған item_NN қолдану
vm_m = Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vm_m)  # қалыпқа келтіруді Lambda қабатымен орындау
model_m = Model(input_item_m, vm_m)                  # модель құру
model_m.summary()                                    # модель құрылымын шығару


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (None, 32)             │        41,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_5 (Lambda)               │ (None, 32)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41,376 (161.62 KB)

 Trainable params: 41,376 (161.62 KB)

 Non-trainable params: 0 (0.00 B)

Фильм моделін алғаннан кейін, фильм ерекшелік векторларының жиынын құруға болады — бұл үшін модельге `item_vecs` сияқты фильм/элемент векторларының жиынын енгізу арқылы болжау жүргізіледі. `item_vecs` — барлық фильм векторларының жиыны. Есіңізде болсын, бір фильм әр жанр үшін жеке вектор ретінде кездеседі. Бұл векторларды оқытылған модельмен қолдану үшін алдын ала масштабтау қажет. Болжау нәтижесінде әр фильм үшін 32 өлшемді ерекшелік векторы алынады.

In [81]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:, i_s:])
print(f"Болжанған барлық фильм ерекшелік векторларының өлшемі: {vms.shape}")

59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Болжанған барлық фильм ерекшелік векторларының өлшемі: (1883, 32)


In [84]:
# Ensure the user vector is properly created
user_vecs, y_vecs = get_user_vecs(user_id, scalerUser.inverse_transform(user_train), item_vecs, user_to_genre)

# Prepare the user vector
user_vec_scaled = user_vecs[0][:num_user_features]
user_vec_scaled = user_vec_scaled.reshape(1, -1)

# Print the shape of the user vector
print("User vector shape:", user_vec_scaled.shape)

# Prepare the item vectors
item_vecs_scaled = scalerItem.transform(item_vecs)
item_vecs_scaled = item_vecs_scaled[:, i_s:]

# Print the shape of the item vectors
print("Item vector shape:", item_vecs_scaled.shape)

# Reshape user vector to match the number of items
scaled_user_vecs = np.tile(user_vec_scaled, (item_vecs_scaled.shape[0], 1))

# Print the shape of scaled user vectors
print("Scaled user vector shape:", scaled_user_vecs.shape)

# Proceed with predictions if everything is correct
predictions = model.predict([scaled_user_vecs, item_vecs_scaled])


User vector shape: (1, 14)
Item vector shape: (1883, 16)
Scaled user vector shape: (1883, 14)
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


Енді әр фильм ерекшелік векторы мен қалған барлық фильм ерекшелік векторлары арасындағы квадраттық қашықтық матрицасын есептейік.

Содан кейін әр жол бойынша минимумды табу арқылы ең жақын фильмді анықтай аламыз. Бұл жерде біз [numpy masked arrays](https://numpy.org/doc/1.21/user/tutorial-ma.html) — маскаланған массивтерді қолданамыз, себебі олар бір фильмнің өзін таңдап алуын болдырмайды. Диагональ бойындағы маскаланған мәндер есептеуге қосылмайды.

# 1-тапсырма: Пайдаланушыға жекелендірілген фильм ұсынымдарын жасау функциясы

## **Функция не істеуі керек:**

**Input:**
- Пайдаланушының ID-сі (мысалы, `uid = 36`)
- Пайдаланушы туралы деректер (мысалы, оның жанрларға деген қалауы немесе бағалау тарихы)
- Фильм/элемент деректері және алдын ала оқытылған модель

**Өңдеу:**
- Берілген пайдаланушы ID-сі бойынша пайдаланушының ерекшелік векторын табу немесе жасау
- Осы векторды деректер жиынындағы әр фильм үшін қайталау — әр фильмге жеке баға болжау үшін
- Енгізілген векторларды модель талап ететін форматқа сәйкестендіріп масштабтау
- Оқытылған модельді қолданып, пайдаланушыға әр фильм қаншалықты ұнайтынын болжау
- Фильмдерді болжау нәтижелері бойынша сұрыптау (ең жоғарғы ұсынылғаннан төменге қарай)

**output:**
- Осы пайдаланушы үшін ұсынылатын ең үздік фильмдердің тізімін (немесе кестесін) шығару
- Төмендегідей мәліметтерді көрсету:
  - Фильм атауы  
  - Жанры  
  - Болжау бойынша баға  
  - (Қосымша) Нақты баға, егер қолжетімді болса

---

**Мысал:**

36-нөмірлі пайдаланушы көбіне экшн және оқиғаға толы фильмдерді көреді. Функцияны орындағанда төмендегідей нәтиже берілуі мүмкін:

| Фильм атауы            | Жанр      | Болжау бағасы |
|------------------------|-----------|----------------|
| Avengers: Endgame      | Action    | 4.9            |
| Mad Max: Fury Road     | Adventure | 4.8            |
| John Wick              | Action    | 4.7            |

In [88]:
import numpy as np
import pandas as pd

def recommend_movies(user_id, user_train, item_vecs, model, user_to_genre, scalerUser, scaler, scalerItem, scaledata, movie_dict, maxcount=10):
    """
    Пайдаланушыға жекелендірілген фильм ұсынымдарын жасау
    """

    # 1. Пайдаланушы векторын алу немесе құру
    user_vecs, y_vecs = get_user_vecs(user_id, scalerUser.inverse_transform(user_train), item_vecs, user_to_genre)

    # 2. Пайдаланушы векторын дайындау
    user_vec_scaled = user_vecs[0][:num_user_features]
    user_vec_scaled = user_vec_scaled.reshape(1, -1)

    # 3. Фильмдер векторын дайындау
    item_vecs_scaled = scalerItem.transform(item_vecs)
    item_vecs_scaled = item_vecs_scaled[:, i_s:]

    scaled_user_vecs = np.tile(user_vec_scaled, (item_vecs_scaled.shape[0], 1))

    # 4. Болжау жасау
    predictions = model.predict([scaled_user_vecs, item_vecs_scaled])# 1. Модель арқылы барлық фильмдер үшін болжау (пайдаланушы осы фильмге қаншалықты жоғары баға беруі мүмкін екенін болжайды)
    sorted_index = np.argsort(predictions.flatten())[::-1]# 2. Болжау нәтижелерін кему ретімен сұрыптау
    sorted_ypu = predictions[sorted_index]

    sorted_movie_ids = sorted_index[sorted_index < len(item_vecs)]# 4. Артық индекстерді алып тастау (индекстер тек бар фильмдердің ішінен алынуы керек)
    sorted_movie_ids = sorted_movie_ids[:maxcount]# 5. Тек ең жақсы ұсынылатын maxcount фильмді таңдаймыз

    # 5. Ұсыныстар тізімін құру
    recommendations = []# Ұсынылатын фильмдердің тізімі
    for i in range(min(maxcount, len(sorted_movie_ids))):
        movie_id = int(item_vecs[sorted_index[i]][0])
        movie_info = movie_dict.get(movie_id, {})

        rec = {
            'Фильм атауы': movie_info.get('title', f"Movie ID {movie_id}"),
            'Жанры': movie_info.get('genres', 'Белгісіз'),
            'Болжамды баға': f"{sorted_ypu[i][0]:.1f}",
        }

        if 'movie rating ave' in movie_info:
            rec['movie rating ave'] = f"{movie_info['movie rating ave']:.1f}"

        recommendations.append(rec)

    return pd.DataFrame(recommendations)

# Вызов функции и вывод результата
recommendations = recommend_movies(
    user_id=36,
    user_train=user_train,
    item_vecs=item_vecs,
    model=model,
    user_to_genre=user_to_genre,
    scalerUser=scalerUser,
    scaler=scaler,
    scalerItem=scalerItem,
    scaledata=scaledata,
    movie_dict=movie_dict,
    maxcount=10
)

print(recommendations)


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
                          Фильм атауы  \
0                   District 9 (2009)   
1                     Sunshine (2007)   
2                    Idiocracy (2006)   
3                    Idiocracy (2006)   
4                   Collateral (2004)   
5        Bourne Supremacy, The (2004)   
6        Bourne Ultimatum, The (2007)   
7            Scanner Darkly, A (2006)   
8               28 Weeks Later (2007)   
9  Hellboy II: The Golden Army (2008)   

                                     Жанры Болжамды баға  
0                  Mystery|Sci-Fi|Thriller           1.0  
1          Adventure|Drama|Sci-Fi|Thriller           1.0  
2         Adventure|Comedy|Sci-Fi|Thriller           0.9  
3         Adventure|Comedy|Sci-Fi|Thriller           0.9  
4              Action|Crime|Drama|Thriller           0.9  
5                    Action|Crime|Thriller           0.9  
6                    Action|Crime|Thriller           0.9  
7  Animation|Drama|Mystery|Sci-Fi|Thr


### 2-Тапсырма: Фильмге ұқсас 10 фильмді табу

Функция жазу — оған фильмнің атауы беріледі, ал нәтиже ретінде **ең ұқсас 10 фильмнің** тізімін қайтарады (векторлық ұқсастыққа негізделген).

---

### Функция талаптары

- **Input:**  
  - `movie_name` (*str*): Ізделетін фильмнің атауы.
  - Қосымша параметрлер: фильм векторлары, метадеректер, ұқсастық пен жанрды есептейтін көмекші функциялар.

- **Процесс:**  
  1. Фильмнің атауы арқылы оның векторлық ұсынуын табу.
  2. Осы фильм мен қалған барлық фильмдердің арасындағы ұқсастықты (мысалы, евклидтік қашықтық) есептеу.
  3. Ұқсастық дәрежесіне қарай фильмдерді сұрыптап, ең ұқсас 10 фильмді таңдау (бастапқы фильмді қоспағанда).
  4. Бастапқы фильм мен табылған фильмдердің атаулары мен жанрларын шығару.

- **Output:**  
  - Кесте немесе тізім, мына ақпаратпен:
    - Енгізілген фильмнің атауы және жанры.
    - Ең ұқсас 10 фильмнің атаулары мен жанрлары.

---

### Мысал

| Бастапқы фильм | Жанры     | Ұқсас фильм     | Жанры     |
|----------------|-----------|------------------|-----------|
| Inception      | Ғылыми-фантастика | Interstellar   | Ғылыми-фантастика |
| Inception      | Ғылыми-фантастика | The Matrix     | Экшн      |
| ...            | ...       | ...              | ...       |



In [55]:
count = 50# Алынатын фильмдер саны (алғашқы 50 фильм)
dim = len(vms)# Барлық фильмдердің саны (вектор саны)
dist = np.zeros((dim,dim))

for i in range(dim):# Екі фильм арасындағы квадраттық қашықтықты есептеу үшін матрица дайындау
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])# i және j фильмдер арасындағы квадраттық қашықтық
        
m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0])) # Нақты бір фильмнің өзі-өзімен салыстырылмауы үшін, диагоналды маскамен жабамыз

disp = [["movie1", "genres", "movie2", "genres"]]# Ұқсас фильм жұптарын шығару үшін тізім дайындау
for i in range(count):
    min_idx = np.argmin(m_dist[i])# i-нші фильмге ең жақын (ұқсас) фильм индексі
    movie1_id = int(item_vecs[i,0])# i-нші фильмнің ID
    movie2_id = int(item_vecs[min_idx,0])# ең ұқсас фильмнің ID
    # Екі фильмнің жанрын алу
    genre1,_  = get_item_genre(item_vecs[i,:], ivs, item_features)
    genre2,_  = get_item_genre(item_vecs[min_idx,:], ivs, item_features)

    disp.append( [movie_dict[movie1_id]['title'], genre1,# Нәтижені тізімге қосу
                  movie_dict[movie2_id]['title'], genre2]
               )
table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow", floatfmt=[".1f", ".1f", ".0f", ".2f", ".2f"])
table

movie1,genres,movie2,genres
Save the Last Dance (2001),Drama,John Q (2002),Drama
Save the Last Dance (2001),Romance,Saving Silverman (Evil Woman) (2001),Romance
"Wedding Planner, The (2001)",Comedy,National Lampoon's Van Wilder (2002),Comedy
"Wedding Planner, The (2001)",Romance,Mr. Deeds (2002),Romance
Hannibal (2001),Horror,Final Destination 2 (2003),Horror
Hannibal (2001),Thriller,"Sum of All Fears, The (2002)",Thriller
Saving Silverman (Evil Woman) (2001),Comedy,Cats & Dogs (2001),Comedy
Saving Silverman (Evil Woman) (2001),Romance,Save the Last Dance (2001),Romance
Down to Earth (2001),Comedy,Joe Dirt (2001),Comedy
Down to Earth (2001),Fantasy,"Haunted Mansion, The (2003)",Fantasy
